### RandomForestClassifier

- 화이트와인 분류 문제

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

- 기본적으로 100개의 결정 트리 사용

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs=-1을 지정하여 모든 CPU 코어 사용
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # 훈련 세트에 대한 점수도 반환
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승
- 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문 -> 더 많은 특성이 훈련에 기여할 기회 얻음
- 과대적합을 줄이고 일반화 성능을 높이는데 도움

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


- OOB 점수 얻기 -> 교차 검증 대신 -> 훈련 세트에 더 많은 샘플을 사용할 수 있다. 

### Extra Trees
- 부트스트랩 샘플을 사용하지 않는 것이 차이점
- 결정 트리를 만들 때 전체 훈련 세트를 사용
- 노드를 분할할 때 무작위로 분할

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### Gradient Boosting
- 기본적으로 깊이가 3인 결정 트리를 100개 사용 -> 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 높은 일반화 성능 기대

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


### Histogram-based Gradient Boosting
- 입력 특성을 256개의 구간으로 나눠 노드 분할 시 최적의 분할을 매우 빠르게 찾음
- n_estimators 대신에 부스팅 반복 횟수를 지정하는 max_iter를 사용

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1) # 랜덤하게 섞을 횟수 지정
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


- 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산
- [특성 중요도, 평균, 표준편차]

In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


- gradient boosting과 비슷하게 조금 더 당도에 집중

In [15]:
hgb.score(test_input, test_target)

0.8723076923076923

In [16]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42) # 히스토그램 기반 그래디언트 부스팅 
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [18]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
